# Translate Document

## Install Libraries

In [4]:
pip install --upgrade google-cloud-translate google-cloud-storage FPDF2 srt

Note: you may need to restart the kernel to use updated packages.


## Set variables

In [21]:
project_id = "jo-vertex-ai-playground-ffyc"
location = "us-central1"
bucket_name = "jo-translation-docs"
input_folder_name = "input"
input_file_name = "KT_session_with_PwC.vtt"
converted_folder_name = "converted"
converted_file_name = "KT_session_with_PwC.pdf"
output_folder_name = "output"
output_file_name = "KT_session_with_PwC_FR.pdf"
source_language_code = "en"
target_language_code = "fr"

## Download VTT document

In [22]:
from google.cloud import storage

def read_gcs_file(bucket_name, input_folder_name, input_file_name):

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(f"{input_folder_name}/{input_file_name}")

    content = blob.download_as_string()
    content = content.decode('utf-8') 

    return content

file_content = read_gcs_file(bucket_name, input_folder_name, input_file_name)
print(file_content)

WEBVTT

afca6962-032a-4634-9de8-45e5964af34d/36-0
00:00:04.380 --> 00:00:08.633
<v Arra Aravindhan>And also going to be doing the
presentation and then we have,</v>

afca6962-032a-4634-9de8-45e5964af34d/36-1
00:00:08.633 --> 00:00:09.250
<v Arra Aravindhan>umm, air.</v>

afca6962-032a-4634-9de8-45e5964af34d/49-0
00:00:09.310 --> 00:00:13.470
<v Arra Aravindhan>Who's the chief architect for CS
PM?</v>

afca6962-032a-4634-9de8-45e5964af34d/63-0
00:00:13.560 --> 00:00:15.730
<v Arra Aravindhan>So we did a security scanning.</v>

afca6962-032a-4634-9de8-45e5964af34d/72-0
00:00:15.800 --> 00:00:20.070
<v Arra Aravindhan>The security scanning is of in
two phases.</v>

afca6962-032a-4634-9de8-45e5964af34d/98-0
00:00:20.080 --> 00:00:24.260
<v Arra Aravindhan>Number one was the CWP, which
was the UH-20.</v>

afca6962-032a-4634-9de8-45e5964af34d/92-0
00:00:23.620 --> 00:00:25.430
<v Sean Hayes>Sorry, Ara, just interrupt you a
second.</v>

afca6962-032a-4634-9de8-45e5964af34d/118-0
00:00:25.440 

## Convert VTT to PDF

In [23]:
from fpdf import FPDF
from google.cloud import storage

def convert_to_pdf(text_content, bucket_name, converted_folder_name, converted_file_name):

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("helvetica", size=12)

    pdf.multi_cell(0, 10, text_content) 
    temp_file_path = "temp_output.pdf"
    pdf.output(temp_file_path)

    # Upload to GCS
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(f"{converted_folder_name}/{converted_file_name}")
    blob.upload_from_filename(temp_file_path)

# Example usage
pdf_file_path = "output.pdf"
convert_to_pdf(file_content, bucket_name, converted_folder_name, converted_file_name)

## Translate function

In [63]:
# Imports the Google Cloud Translation library
from google.cloud import translate_v3

# Initialize Translation client
def translate_document(    
    bucket_name,
    converted_folder_name,
    converted_file_name,
    location,
    project_id,    
    source_language_code,
    target_language_code,
    output_folder_name,
    output_file_name
) -> translate_v3.TranslationServiceClient:

    client = translate_v3.TranslationServiceClient()

    location = location

    parent = f"projects/{project_id}/locations/{location}"
    
    document_input_config = {
        "gcs_source": {
            "input_uri": f"gs://{bucket_name}/{converted_folder_name}/{converted_file_name}"
        },
        "mime_type": "application/pdf"
    }
        
    document_output_config = {
        "gcs_destination": {
            "output_uri_prefix": f"gs://{bucket_name}/{output_folder_name}/"
        },
        "file_name": output_file_name
    }
  
    response = client.translate_document(
        request={
            "parent": parent,
            "source_language_code": source_language_code,
            "target_language_code": target_language_code,
            "document_input_config": document_input_config,
            "document_output_config": document_output_config,
            "is_translate_native_pdf_only": True
        }
    )
    
    print(response)

    return response

## Call function

In [64]:
translated_document = translate_document(
    bucket_name,
    converted_folder_name,
    converted_file_name,
    location,
    project_id,    
    source_language_code,
    target_language_code,
    output_folder_name,
    output_file_name
)

ValueError: Protocol message DocumentOutputConfig has no "file_name" field.